In [25]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1,
                  streaming=True,
                  callbacks=[StdOutCallbackHandler])

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a word-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food."),   
])

chef_chain = chef_prompt | chat

In [ ]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegeterain. You find alternative ingredients and explain their preparation. YOu don't radically modifiy the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}")
])
veg_chain = veg_chef_prompt | chat
final_chain = {"recipe": chef_chain} | veg_chain
final_chain.invoke({
    "cuisine" : "indian",
})